In [1]:
# Import necessary libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ad-clicks/ad_clicks.csv
/kaggle/input/clicks/clicks.csv
/kaggle/input/musclehub-abtest/purchases.csv
/kaggle/input/musclehub-abtest/fitness_tests.csv
/kaggle/input/musclehub-abtest/visits.csv
/kaggle/input/musclehub-abtest/interviews.txt
/kaggle/input/musclehub-abtest/applications.csv


#  **Testing for Shoefly.com**

Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures. Some questions that will be answered are the following:
1. How many views came from each utm_source?
1. Was there a difference in click rates for each source?
1. Were approximately the same number of people shown both ads?
1. Product manager for the A/B test thinks that the clicks might have changed by day of the week.
1. Do you recommend that your company use Ad A or Ad B?


In [2]:
# Fill in the line below: Specify the path of the CSV file to read
my_filepath = "../input/ad-clicks/ad_clicks.csv"

# Fill in the line below: Read the file into a variable my_data
ad_clicks = pd.read_csv(my_filepath)

In [3]:
# To preview the dataframe to see what data we are working with
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [4]:
# How many views (ie, rows of the table) came from each utm_source?
# Can use any of the columns because using the count() to find value of rows of the table. 
# Recall that added .reset_index() to return result from a series to a dataframe.

ad_clicks.groupby('utm_source')\
    .user_id.count()\
    .reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


In [5]:
# Filter the data to remove NaN values since those mean the user never clicked the ads 
# and add a new column is_click. ~ is a NOT operator to invert the results of .isnull() 
# since .isnull() will return True for NaN and we want it to return False

ad_clicks['is_click'] = ~ad_clicks\
   .ad_click_timestamp.isnull()

# We want to know the percent of people who clicked on ads from each utm_source.
# Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. 
# Save your answer to the variable clicks_by_source.
clicks_by_source = ad_clicks\
   .groupby(['utm_source',
             'is_click'])\
   .user_id.count()\
   .reset_index()

clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [6]:
# Pivot the data so that columns are is_click (True or False), the index is utm_source, 
# and the values are user_id

clicks_pivot = clicks_by_source\
   .pivot(index='utm_source',
          columns='is_click',
          values='user_id')\
   .reset_index()

# Was there a difference in click rates for each source? Find the percent clicked and add 
# a new columm in the clicks_pivot table

clicks_pivot['percent_clicked'] = \
   clicks_pivot[True] / \
   (clicks_pivot[True] + 
    clicks_pivot[False])\
    *100
    
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,175,80,31.372549
1,facebook,324,180,35.714286
2,google,441,239,35.147059
3,twitter,149,66,30.697674


We found that there was a difference in click rates for each source. Facebook ads had the highest percentage at 35.71% closely followed by
Google ads at 35.15%. The lowest performing ads clicked were from Twitter.

In [7]:
# Analyzing an A/B test. Column experimental_group tells us whether the user was shown Ad A or Ad B. 
# Using the column is_click, check to see if a greater percentage of users clicked on Ad A or Ad B. 

clicks_exp_pivot = ad_clicks\
   .groupby(['experimental_group',
             'is_click'])\
   .user_id.count()\
   .reset_index()\
   .pivot(
     index='experimental_group',
     columns='is_click',
     values='user_id')\
   .reset_index()\

clicks_exp_pivot['percent_clicked'] = \
   clicks_exp_pivot[True] / \
   (clicks_exp_pivot[True] + 
    clicks_exp_pivot[False])\
    *100
    
clicks_exp_pivot

is_click,experimental_group,False,True,percent_clicked
0,A,517,310,37.484885
1,B,572,255,30.834341


A user who was given Ad A was more likely to click the ad at 37.48% than a user who was given Ad B. 

In [8]:
# Also check to see if clicks might have changed by day of the week. Do you recommend that 
# your company use Ad A or Ad B?
# Creating DataFrame for a_clicks.
a_clicks = ad_clicks[
   ad_clicks.experimental_group == 'A']

a_clicks_pivot = a_clicks\
  .groupby(['is_click','day'])\
  .user_id.count()\
  .reset_index()\
  .pivot(
     index = 'day',
     columns = 'is_click',
     values = 'user_id'
   )\
   .reset_index()

a_clicks_pivot['percent_clicked'] = \
   a_clicks_pivot[True] / \
   (a_clicks_pivot[True] + 
    a_clicks_pivot[False])\
    *100

a_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,38.053097
1,2 - Tuesday,76,43,36.134454
2,3 - Wednesday,86,38,30.645161
3,4 - Thursday,69,47,40.517241
4,5 - Friday,77,51,39.843750
5,6 - Saturday,73,45,38.135593
6,7 - Sunday,66,43,39.449541


In [9]:
# Creating DataFrame for b_clicks.
b_clicks = ad_clicks[
   ad_clicks.experimental_group == 'B']

b_clicks_pivot = b_clicks\
  .groupby(['is_click','day'])\
  .user_id.count()\
  .reset_index()\
  .pivot(
     index = 'day',
     columns = 'is_click',
     values = 'user_id'
   )\
   .reset_index()

b_clicks_pivot['percent_clicked'] = \
   b_clicks_pivot[True] / \
   (b_clicks_pivot[True] + 
    b_clicks_pivot[False])\
    *100

b_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,28.318584
1,2 - Tuesday,74,45,37.815126
2,3 - Wednesday,89,35,28.225806
3,4 - Thursday,87,29,25.000000
4,5 - Friday,90,38,29.687500
5,6 - Saturday,76,42,35.593220
6,7 - Sunday,75,34,31.192661


During the days of the week, the performance of Ad A outpaced that of Ad B. Ad B, in particular only performed better than Ad A on Tuesday.

To summarize the results, we learned that Ad A performed better than Ad B regardless of the day of the week the ad was shown. We also learned that ads on Facebook were clicked at a higher rate than its competitors. Therefore, I would recommend that the company run Ad A instead of Ad B and allocate more resources into running the ad on Facebook and Google than the other platforms to maximize user traffic into ShoeFly.com.
